In [25]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Explore data

In [26]:
train_data = pd.read_csv("input/nlp-getting-started/train.csv")
train_data

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [48]:
train_data.columns

Index(['id', 'keyword', 'location', 'text', 'target'], dtype='object')

In [49]:
train_data.describe()

,id,target
count,7613.000000,7613.00000
mean,5441.934848,0.42966
std,3137.116090,0.49506
min,1.000000,0.00000
25%,2734.000000,0.00000
50%,5408.000000,0.00000
75%,8146.000000,1.00000
max,10873.000000,1.00000


* **Note** we note that keyword is not a real good classifier to obtain our target, and also we note that location can be an important feature to add with the text to predict our target.

# Preprocessing data

## Preprocessing location

In [4]:
featuretest_data=pd.read_pickle("location.pkl")
featuretest_data.sample(20)

,id,keyword,location,text,target,geolocation,geolocation_converted
2072,2974,dead,somewhere in Portugal,If itÛªs a war you came to see you will never...,0,NaN,False
505,730,attacked,"Livingston, IL U.S.A.",Christian Attacked by Muslims at the Temple Mo...,1,NaN,False
2282,3276,demolish,Hooters on Peachtree,NOTHING YOU MIDGET I WILL DEMOLISH YOU SHOW SO...,0,NaN,False
4527,6436,injured,"Srinagar,Kashmir",Militants attack police post in Udhampur; 2 SP...,1,"(Kashmir Valley, Shalteng, Srīnagar, Srinagar ...",True
1941,2792,curfew,"Adelaide, Australia",INFO U. CLD: SCT012 BKN025. EXP INST APCH. RWY...,0,"(Adelaide, Adelaide City Council, South Austra...",True
6293,8992,stretcher,CamaquÌ£/Pelotas,Somebody get the doctor I'm feelin' pretty poo...,0,NaN,False
1473,2122,catastrophe,"New Brunswick, NJ",God bless catastrophe,0,"(New Brunswick, Middlesex County, New Jersey, ...",True
6598,9450,terrorism,USA,Preacher faces UK terrorism charges http://t.c...,1,"(United States, (39.7837304, -100.445882))",True
412,598,arsonist,ss,@_301DC @Cloudy_goldrush i hate white people mo,1,"(South Sudan جنوب السودان, (7.8699431, 29.6667...",True
5355,7643,pandemonium,California,Truly a scene of chaos unprecedented in frenzy...,1,"(California, United States, (36.7014631, -118....",True


In [14]:
#Show detailed geolocation object from the first entry
featuretest_data.iloc[0]["geolocation"].raw

{'place_id': 243595575,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright',
 'osm_type': 'relation',
 'osm_id': 162378,
 'lat': '52.4796992',
 'lon': '-1.9026911',
 'class': 'boundary',
 'type': 'administrative',
 'place_rank': 16,
 'importance': 0.6734272656139734,
 'addresstype': 'city',
 'name': 'Birmingham',
 'display_name': 'Birmingham, West Midlands Combined Authority, England, United Kingdom',
 'address': {'city': 'Birmingham',
  'ISO3166-2-lvl8': 'GB-BIR',
  'state_district': 'West Midlands Combined Authority',
  'state': 'England',
  'ISO3166-2-lvl4': 'GB-ENG',
  'country': 'United Kingdom',
  'country_code': 'gb'},
 'boundingbox': ['52.3810530', '52.6087058', '-2.0336486', '-1.7288417']}

In [17]:
## Extract country from geolocation
from functools import partial

def _geolocation_to_scope(geolocation, scope:str):
    if geolocation is None or pd.isna(geolocation) or scope not in geolocation.raw["address"]:
        return np.nan
    return geolocation.raw["address"][scope]

def extract_from_geolocation(df:pd.DataFrame, scope:str):
    df[scope]=df["geolocation"].map(partial(_geolocation_to_scope,scope=scope))
    return df
extract_from_geolocation(featuretest_data,scope="country_code")

,id,keyword,location,text,target,geolocation,geolocation_converted,city,country,country_code
31,48,ablaze,Birmingham,@bbcmtd Wholesale Markets ablaze http://t.co/l...,1,"(Birmingham, West Midlands Combined Authority,...",True,Birmingham,United Kingdom,gb
32,49,ablaze,Est. September 2012 - Bristol,We always try to bring the heavy. #metal #RT h...,0,NaN,False,NaN,NaN,NaN
33,50,ablaze,AFRICA,#AFRICANBAZE: Breaking news:Nigeria flag set a...,1,"(Africa, (11.5024338, 17.7578122))",True,NaN,NaN,NaN
34,52,ablaze,"Philadelphia, PA",Crying out for more! Set me ablaze,0,"(Philadelphia, Philadelphia County, Pennsylvan...",True,Philadelphia,United States,us
35,53,ablaze,"London, UK",On plus side LOOK AT THE SKY LAST NIGHT IT WAS...,0,"(London, Greater London, England, United Kingd...",True,London,United Kingdom,gb
...,...,...,...,...,...,...,...,...,...,...
7575,10826,wrecked,TN,On the bright side I wrecked http://t.co/uEa0t...,0,"(تونس, (33.8439408, 9.400138))",True,NaN,تونس,tn
7577,10829,wrecked,#NewcastleuponTyne #UK,@widda16 ... He's gone. You can relax. I thoug...,0,NaN,False,NaN,NaN,NaN
7579,10831,wrecked,"Vancouver, Canada",Three days off from work and they've pretty mu...,0,"(Vancouver, Metro Vancouver Regional District,...",True,Vancouver,Canada,ca
7580,10832,wrecked,London,#FX #forex #trading Cramer: Iger's 3 words tha...,0,"(London, Greater London, England, United Kingd...",True,London,United Kingdom,gb


## Missing Values


### Missing location

- Sample randomly from the country with weights based on occurences on the dataset

In [27]:
train_data_without_location = train_data[train_data["location"].isna()]
train_data_without_location

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [30]:
# Concatenate dataframe without location + preprocessing location (executed only on existing locations)
all_columns = set(featuretest_data.columns).union(train_data_without_location.columns)

# Concatenate dataframes and fill missing values with None
train_data_preprocessed_location = pd.concat([featuretest_data.reindex(columns=all_columns),
                             train_data_without_location.reindex(columns=all_columns)], ignore_index=True)
train_data_preprocessed_location

,city,text,target,location,country_code,id,keyword,geolocation_converted,country,geolocation
0,Birmingham,@bbcmtd Wholesale Markets ablaze http://t.co/l...,1,Birmingham,gb,48,ablaze,1.0,United Kingdom,"(Birmingham, West Midlands Combined Authority,..."
1,NaN,We always try to bring the heavy. #metal #RT h...,0,Est. September 2012 - Bristol,gb,49,ablaze,0.0,NaN,NaN
2,NaN,#AFRICANBAZE: Breaking news:Nigeria flag set a...,1,AFRICA,ss,50,ablaze,1.0,NaN,"(Africa, (11.5024338, 17.7578122))"
3,Philadelphia,Crying out for more! Set me ablaze,0,"Philadelphia, PA",us,52,ablaze,1.0,United States,"(Philadelphia, Philadelphia County, Pennsylvan..."
4,London,On plus side LOOK AT THE SKY LAST NIGHT IT WAS...,0,"London, UK",gb,53,ablaze,1.0,United Kingdom,"(London, Greater London, England, United Kingd..."
...,...,...,...,...,...,...,...,...,...,...
7608,NaN,Two giant cranes holding a bridge collapse int...,1,NaN,NaN,10869,NaN,NaN,NaN,NaN
7609,NaN,@aria_ahrary @TheTawniest The out of control w...,1,NaN,NaN,10870,NaN,NaN,NaN,NaN
7610,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1,NaN,NaN,10871,NaN,NaN,NaN,NaN
7611,NaN,Police investigating after an e-bike collided ...,1,NaN,NaN,10872,NaN,NaN,NaN,NaN


In [31]:
missing_indices= train_data_preprocessed_location["country_code"].isna()
number_missing=missing_indices.sum()
class_counts = train_data_preprocessed_location["country_code"].value_counts(normalize=True)
random_classes = np.random.choice(class_counts.index, size=number_missing, p=class_counts.values)

# Fill missing values with randomly selected values
train_data_preprocessed_location.loc[missing_indices,"country_code"] = random_classes

In [32]:
train_data_preprocessed_location

,city,text,target,location,country_code,id,keyword,geolocation_converted,country,geolocation
0,Birmingham,@bbcmtd Wholesale Markets ablaze http://t.co/l...,1,Birmingham,gb,48,ablaze,1.0,United Kingdom,"(Birmingham, West Midlands Combined Authority,..."
1,NaN,We always try to bring the heavy. #metal #RT h...,0,Est. September 2012 - Bristol,gb,49,ablaze,0.0,NaN,NaN
2,NaN,#AFRICANBAZE: Breaking news:Nigeria flag set a...,1,AFRICA,ss,50,ablaze,1.0,NaN,"(Africa, (11.5024338, 17.7578122))"
3,Philadelphia,Crying out for more! Set me ablaze,0,"Philadelphia, PA",us,52,ablaze,1.0,United States,"(Philadelphia, Philadelphia County, Pennsylvan..."
4,London,On plus side LOOK AT THE SKY LAST NIGHT IT WAS...,0,"London, UK",gb,53,ablaze,1.0,United Kingdom,"(London, Greater London, England, United Kingd..."
...,...,...,...,...,...,...,...,...,...,...
7608,NaN,Two giant cranes holding a bridge collapse int...,1,NaN,au,10869,NaN,NaN,NaN,NaN
7609,NaN,@aria_ahrary @TheTawniest The out of control w...,1,NaN,ph,10870,NaN,NaN,NaN,NaN
7610,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1,NaN,au,10871,NaN,NaN,NaN,NaN
7611,NaN,Police investigating after an e-bike collided ...,1,NaN,us,10872,NaN,NaN,NaN,NaN


### Missing keywords

In [33]:
import pandas as pd
import importlib 
import preprocess.keywords as kw
importlib.reload(kw)
layer=kw.PreprocessKeywords()
layer.preprocess(train_data_preprocessed_location)


,city,text,target,location,country_code,id,keyword,geolocation_converted,country,geolocation,keyword_prep
0,Birmingham,@bbcmtd Wholesale Markets ablaze http://t.co/l...,1,Birmingham,gb,48,ablaze,1.0,United Kingdom,"(Birmingham, West Midlands Combined Authority,...",ablaze
1,NaN,We always try to bring the heavy. #metal #RT h...,0,Est. September 2012 - Bristol,gb,49,ablaze,0.0,NaN,NaN,ablaze
2,NaN,#AFRICANBAZE: Breaking news:Nigeria flag set a...,1,AFRICA,ss,50,ablaze,1.0,NaN,"(Africa, (11.5024338, 17.7578122))",ablaze
3,Philadelphia,Crying out for more! Set me ablaze,0,"Philadelphia, PA",us,52,ablaze,1.0,United States,"(Philadelphia, Philadelphia County, Pennsylvan...",ablaze
4,London,On plus side LOOK AT THE SKY LAST NIGHT IT WAS...,0,"London, UK",gb,53,ablaze,1.0,United Kingdom,"(London, Greater London, England, United Kingd...",ablaze
...,...,...,...,...,...,...,...,...,...,...,...
7608,NaN,Two giant cranes holding a bridge collapse int...,1,NaN,au,10869,NaN,NaN,NaN,NaN,collapse
7609,NaN,@aria_ahrary @TheTawniest The out of control w...,1,NaN,ph,10870,NaN,NaN,NaN,NaN,fires
7610,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1,NaN,au,10871,NaN,NaN,NaN,NaN,Hawaii
7611,NaN,Police investigating after an e-bike collided ...,1,NaN,us,10872,NaN,NaN,NaN,NaN,collided
